In [1]:
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
import glob

In [2]:
# mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [10]:
clips_filenames = glob.glob("/home/rupanshu/Desktop/Content/TrAIner/clips/*")

In [11]:
clips_filenames = sorted(clips_filenames)

In [12]:
clips_np = []
clips_poses = []

In [13]:
def get_reqd_landmarks(pose):
    try:
        pose_landmarks = [[res.x, res.y] for res in pose.pose_landmarks.landmark]
        reqd_landmarks = np.array(pose_landmarks)[[11, 12, 23, 24, 25, 26]]
        return reqd_landmarks
    except:
        return None

In [14]:
clips_filenames = clips_filenames[18:]

In [15]:
with mp_pose.Pose(min_detection_confidence=0.2, min_tracking_confidence=0.2) as pose:
    for idx in range(len(clips_filenames)):
        filename = clips_filenames[idx]
#         print(filename)
        stream = cv2.VideoCapture(filename)
        print(f'{idx} of {(len(clips_filenames)-1)}', end='\r')

        clip_np = []
        poses   = []

        while stream.isOpened():
            ret, frame = stream.read()
            
            if ret is False or (len(np.unique(frame.ravel())) == 1 and np.unique(frame.ravel()) == 0):
                break
            
            extracted_pose = pose.process(frame)
            if extracted_pose is None:
                continue

            clip_np.append(frame)
            poses.append(extracted_pose)

        clips_np.append(clip_np)
        clips_poses.append(poses)

In [16]:
clips_landmarks = []

In [17]:
for i in range(len(clips_poses)):
    landmarks = []
    for j in range(len(clips_poses[i])):
        landmark = get_reqd_landmarks(clips_poses[i][j])
        
        if landmark is not None:
            landmarks.append(landmark)
    clips_landmarks.append(landmarks)

In [18]:
import pickle

In [19]:
len(clips_landmarks)

9

In [20]:
with open('last_9_landmarks.pkl', 'wb') as f:
    pickle.dump(clips_landmarks, f)

In [35]:
max_frames = -1

for i in range(len(clips_landmarks)):
#     print(i+1, len(clips_landmarks[i]))
    max_frames = max(len(clips_landmarks[i]), max_frames)

In [37]:
for clip_idx in range(len(clips_landmarks)):
    delta = max_frames - len(clips_landmarks[clip_idx])
    
    to_fill = clips_landmarks[clip_idx][-1]
    
    for i in range(delta):
        clips_landmarks[clip_idx].append(to_fill)

In [43]:
np.save("final_data_label1.npy", np.array(clips_landmarks))